# CW2

By: Carissa Ying Geok Teng  
Matric No.: A0205190R

In [1]:
import numpy as np
import copy
import cv2
from scipy.linalg import hadamard

import pdb

## Helper Functions

In [2]:
def old_am_power(a):
    pa = 0.0 
    dim1 = a.shape
    if len(dim1)==2:
        sz = dim1[0] * dim1[1] 
        for i in range(dim1[0]):
            for j in range(dim1[1]):
                pa += a[i,j]**2
    elif len(dim1)==3:
        sz = dim1[0] * dim1[1] * dim1[2]
        for i in range(dim1[0]):
            for j in range(dim1[1]):
                for k in range(dim1[2]):
                    pa += a[i,j,k]**2
    pa = pa / sz
    return pa

def old_am_energy(a):
    pa = 0.0 
    dim1 = a.shape
    if len(dim1)==2:
        sz = dim1[0] * dim1[1] 
        for i in range(dim1[0]):
            for j in range(dim1[1]):
                pa += a[i,j]**2
    elif len(dim1)==3:
        sz = dim1[0] * dim1[1] * dim1[2]
        for i in range(dim1[0]):
            for j in range(dim1[1]):
                for k in range(dim1[2]):
                    pa += a[i,j,k]**2
    return pa

#Hadamard helper functions

int2bin = lambda x, n: format(x, 'b').zfill(n)

def horder(b,nn): 
    jj = int2bin(b,nn)
    kk = ''
    for j in range(nn): 
        kk = kk+jj[nn-1-j] 
    
    kkk=np.zeros(nn) 
    kkk[0] = kk[0] 
    for j in range(1,nn):
        kkk[j] = int(kkk[j-1]) ^ int(kk[j]) 
        
    k=0
    for j in range(nn):
        k = k + int(kkk[j]) * 2**(nn-1-j)  

    return int(k)

def ordhad(n): 
    h = hadamard(n)
    hh = hadamard(n)
    nn = np.log2(n)
    for i in range(n):
        k = horder(int(i) , int(nn)) 
        hh[k][:] = h[i][:]

    return hh

In [3]:
def show_image(img, name):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.waitKey(0)

## Q3

In [5]:
# convert to binary
def threshold(edges, t):
    edges = np.copy(edges)
    edges[edges < t] = 0
    edges[edges >= t] = 1
    return edges

# circle detection
def detect_hough_circles(img, r_min=50, r_max=200, theta=360, detect=1, local_max_area=5):
    edges = cv2.Canny(img, 50, 150, apertureSize = 3)
    edges = threshold(edges, (np.max(edges)-np.min(edges))/2)
    
    m, n = edges.shape
    r_range = np.arange(r_max-r_min)
    theta_range = np.arange(theta)
    
    # optimisation of circle equation
    c_x = np.transpose(np.tile(r_range+r_min, (len(theta_range), 1))) * np.cos(theta_range*np.pi/180)
    c_y = np.transpose(np.tile(r_range+r_min, (len(theta_range), 1))) * np.sin(theta_range*np.pi/180)
    # map to center
    c_x = (c_x + r_max).astype("uint8")
    c_y = (c_y + r_max).astype("uint8")
    c = np.zeros((2*r_max, 2*r_max, r_max-r_min), dtype="uint8")
    for r in r_range:
        for theta in theta_range:
            c[c_x[r, theta], c_y[r, theta], r] = 1
            
    e = []
    for e_x in range(m):
        for e_y in range(n):
            if edges[e_x, e_y] == 1:
                e.append((e_x, e_y))
    # count
    a = np.zeros((m+2*r_max, n+2*r_max, r_max-r_min), dtype="uint8")
    for e_x, e_y in e:
        a[e_x:e_x+2*r_max, e_y:e_y+2*r_max, :] += c
    
    local_max = []
    for i in range(detect):
        center_x, center_y, r = np.unravel_index(a.argmax(), a.shape)
        local_max.append((center_x-r_max, center_y-r_max, r+r_min))
        for j in range(-local_max_area, local_max_area):
            for k in range(-local_max_area, local_max_area):
                for l in range(-local_max_area, local_max_area):
                    a[center_x+j, center_y+k, r+l] = 0
    return local_max

def plot_circles(img, circles, line_weight=100, colour=0):
    m, n = img.shape
    new_img = np.copy(img)
    for x, y, r in circles:
        for i in range(m):
            for j in range(n):
                e = (i-x)**2 + (j-y)**2 - r**2
                if e < line_weight and e > -line_weight:
                    new_img[i, j] = colour
    return new_img

In [20]:
img = cv2.imread("./images/95103cv.png", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, dsize=(411, 290))
res = detect_hough_circles(img, r_min=1, r_max=100, detect=3, local_max_area=2)

In [21]:
circles = plot_circles(img, res, 100, colour=255)
show_image(circles, "circles")

## Q4

In [38]:
def ButterworthLowPass(m, n, D0, order):
    filter = np.zeros((m, n))
    # normalized cut_off frequency is mapped to real index
    D0 = D0 * min(m,n) / 2
    order = 2 * order
    for i in range(m):
        for j in range(n):
            d = ( (i-m/2)**2 + (j-n/2)**2 )**0.5
            filter[i,j]= 1 / ( 1 + (d/D0)**order )
            
    return filter

def bandpass_filter(img, low=0.1, high=0.4, order=3):
    m, n = img.shape
    '''YOUR CODE HERE'''
    ft = np.fft.fft2(img)
    ft_shift = np.fft.fftshift(ft)
    ft_shift_abs = np.abs(ft_shift)
    ft_shift_phase = np.angle(ft_shift) 
    
    # filtering
    # low is the lowest frequency allowed -> bound of high-pass filter
    # high is the highest frequency allowed -> bound of low-pass filter
    low_pass_f = ButterworthLowPass(m, n, high, order)
    if low == 0:
        high_pass_f = np.ones((m, n))
    else:
        high_pass_f = 1 - ButterworthLowPass(m, n, low, order)
    band_pass_f = np.multiply(low_pass_f, high_pass_f) 
    ft_shift_abs = np.multiply(ft_shift_abs, band_pass_f) 
    
    # rebuild image
    z = np.multiply (ft_shift_abs , np.exp((1j)*(ft_shift_phase)))
    ift_shift = np.fft.ifftshift(z)
    filt_img = np.fft.ifft2(ift_shift)
    filt_img = np.uint8(np.abs(filt_img))
    
    # power
    filt_power = old_am_power(filt_img)
    '''END OF YOUR CODE'''
        
    return filt_img, filt_power


In [39]:
img = cv2.imread("./images/01a_amusementpark.jpg", 0)
show_image(img, "img")
print(old_am_power(img))
filt_img, filt_power = bandpass_filter(img, 0.1, 0.4)
show_image(filt_img, "img")
print(filt_power)

14284.917959538778
19.234844348617987


In [6]:
img = cv2.imread("./images/01a_amusementpark.jpg", 0)
show_image(img, "img")
print(old_am_power(img))
filt_img, filt_power = bandpass_filter(img, 0.3, 0.4)
show_image(filt_img, "img")
print(filt_power)

14284.917959538778
0.361262653945633


In [7]:
img = cv2.imread("./images/JASDF-1111_.jpg", cv2.IMREAD_GRAYSCALE)
show_image(img, "img")
print(old_am_power(img))
filt_img, filt_power = bandpass_filter(img, 0.1, 0.4)
show_image(filt_img, "img")
print(filt_power)

42251.40167766667
44.85139383333333


In [8]:
img = cv2.imread("./images/JASDF-1111_.jpg", cv2.IMREAD_GRAYSCALE)
show_image(img, "img")
print(old_am_power(img))
filt_img, filt_power = bandpass_filter(img, 0.3, 0.4)
show_image(filt_img, "img")
print(filt_power)

42251.40167766667
3.2559963333333335


## Q5

In [8]:
img = cv2.imread("./images/BBtpZbO.jpg", cv2.IMREAD_GRAYSCALE)
power = old_am_power(img)
filt_img_1, filt_power_1 = bandpass_filter(img, 0, 0.1)
print([0, 0.1], filt_power_1, filt_power_1/power*100)
show_image(filt_img_1, "img")
filt_img_2, filt_power_2 = bandpass_filter(img, 0.1, 0.3)
print([0, 0.1], filt_power_2, filt_power_2/power*100)
show_image(filt_img_2, "img")
filt_img_3, filt_power_3 = bandpass_filter(img, 0.3, 0.7)
print([0, 0.1], filt_power_3, filt_power_3/power*100)
show_image(filt_img_3, "img")

final_img = cv2.addWeighted(filt_img_1, 1, filt_img_2, 1, 0)
final_img = cv2.addWeighted(final_img, 1, filt_img_3, 1, 0)
show_image(final_img, "img")

[0, 0.1] 10223.787472396025 83.2294587720503
[0, 0.1] 486.1164025119617 3.9573597544477117
[0, 0.1] 495.31203418292233 4.03221923769769


In [100]:
show_image(filt_img, "test")

In [9]:
img = cv2.imread("./images/djzam_nat_defect_002_2g_8.bmp", 0)
power = old_am_power(img)
filt_img_1, filt_power_1 = bandpass_filter(img, 0, 0.1)
print([0, 0.1], filt_power_1, filt_power_1/power*100)
show_image(filt_img_1, "img")
filt_img_2, filt_power_2 = bandpass_filter(img, 0.1, 0.3)
print([0, 0.1], filt_power_2, filt_power_2/power*100)
show_image(filt_img_2, "img")
filt_img_3, filt_power_3 = bandpass_filter(img, 0.3, 0.7)
print([0, 0.1], filt_power_3, filt_power_3/power*100)
show_image(filt_img_3, "img")

final_img = cv2.addWeighted(filt_img_1, 1, filt_img_2, 1, 0)
final_img = cv2.addWeighted(final_img, 1, filt_img_3, 1, 0)
show_image(final_img, "img")

[0, 0.1] 27708.43051147461 99.34534715352895
[0, 0.1] 5.0626220703125 0.018151441196715638
[0, 0.1] 1.0201568603515625 0.003657653485667836


## Q6

In [10]:
def idealLowPass(m, n, d_0):
    filter = np.ones((m, n), dtype=np.uint8)
    d_0 = min(m, n) / 2 * d_0
    for i in range(m):
        for j in range(n):
            if ((i-m/2)**2 + (j-n/2)**2)**0.5 >= d_0:
                filter[i,j]= 0
            
    return filter

def bandreject_filter(img, low=0.6, high=0.9):
    m, n = img.shape
    m = 2 ** int(np.log2(min(m, n)))
    '''YOUR CODE HERE'''
    h = ordhad(m)
    w = np.matmul(h, np.matmul(img,h))
    
    # filter
    low_pass_filter = idealLowPass(m*2, m*2, high)[m:, m:]
    if low == 0:
        high_pass_filter = np.ones((m, m), dtype="uint8")
    else:
        high_pass_filter = 1 - idealLowPass(m*2, m*2, low)[m:, m:]
    band_pass_f = np.multiply(low_pass_filter, high_pass_filter)
    band_reject_f = 1 - band_pass_f
    filt_w = np.multiply(w, band_reject_f)
    
    # rebuild image
    filt_img = np.dot(np.matmul(h, np.matmul(filt_w, h)), 1/m**2)
    filt_img = filt_img.astype("uint8")
    
    # power
    filt_power = old_am_power(filt_img)
    '''END OF YOUR CODE'''
    
    return filt_img, filt_power

In [11]:
img = cv2.imread("./images/IMG_0358.JPG", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, dsize=(256, 256))
power = old_am_power(img)
filt_img_1, filt_power_1 = bandreject_filter(img, 0.6, 0.9)
print([0, 0.1], filt_power_1, filt_power_1/power*100)
show_image(filt_img_1, "img")
filt_img_2, filt_power_2 = bandreject_filter(img, 0.4, 0.6)
print([0, 0.1], filt_power_2, filt_power_2/power*100)
show_image(filt_img_2, "img")


[0, 0.1] 15769.0234375 98.16375485209065
[0, 0.1] 15794.775405883789 98.32406344199187


In [12]:
img = cv2.imread("./images/06600600u.bmp", cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, dsize=(256, 256))
power = old_am_power(img)
filt_img_1, filt_power_1 = bandreject_filter(img, 0.6, 0.9)
print([0, 0.1], filt_power_1, filt_power_1/power*100)
show_image(filt_img_1, "img")
filt_img_2, filt_power_2 = bandreject_filter(img, 0.4, 0.6)
print([0, 0.1], filt_power_2, filt_power_2/power*100)
show_image(filt_img_2, "img")

[0, 0.1] 17428.4168548584 98.12726358493491
[0, 0.1] 17495.519760131836 98.50507325794867
